File này: Với các điểm 3D được chuyển đổi trong ROI (output của file project_to_3d), dựa vào output này,   
ta nhìn vào ảnh point cloud để giữ lại các điểm trong ROI.

Hàm phía dưới là để xử lý với 1 ảnh

In [11]:
# Ý tưởng:
# 1 Load scene point cloud .ply
# 2 load ROI point cloud (Xyz)
# 3 với mỗi điểm ROI -> tìm điểm gần nhất trong scene (KDTree Search)
# 4 Giữ các điểm trong scene thuộc ROI

# filter_ply.py

import numpy as np
import open3d as o3d

class PointCloudFilter:
    def __init__(self, distance_threshold=0.01):
        """
        @param distance_threshold: Ngưỡng để xác định điểm nào thuộc ROI (m)
        """
        self.distance_threshold = distance_threshold

        # ===== Extrinsics (Camera -> World) =====
        self.R = np.array([
            [0.9999898076057434, -0.00020347206736914814, -0.004507721401751041],
            [0.00018898719281423837, 0.9999948143959045, -0.0032135415822267532],
            [0.004508351907134056, 0.003212657058611512, 0.9999846816062927]
        ])
        self.T = np.array([-0.05905, 8.67399e-05, 0.00041])

        # Tạo extrinsic matrix 4x4
        self.extrinsic = np.eye(4)
        self.extrinsic[:3, :3] = self.R
        self.extrinsic[:3, 3] = self.T

        # ===== Flip Matrix (World -> PLY Coordinate) =====
        self.flip_mat = np.array([
            [1,  0,  0],
            [0, -1,  0],
            [0,  0, -1]
        ])

    def align_roi_to_scene(self, roi_pcd):
        """
        Áp dụng Extrinsic + Flip để đưa ROI vào hệ tọa độ của Scene PLY.
        """
        roi_points = np.asarray(roi_pcd.points)
        roi_points_h = np.hstack((roi_points, np.ones((roi_points.shape[0], 1))))
        
        # Extrinsic transform
        roi_world = (self.extrinsic @ roi_points_h.T).T[:, :3]

        # Flip to PLY coordinate
        roi_aligned = (self.flip_mat @ roi_world.T).T

        roi_pcd.points = o3d.utility.Vector3dVector(roi_aligned)
        return roi_pcd

    def filter_scene_with_roi(self, scene_ply_path, roi_pcd):
        """
        @param scene_ply_path: đường dẫn file PLY đầy đủ
        @param roi_pcd: Open3D ROI PointCloud (chưa transform)
        @return: filtered_point_cloud (Open3D)
        """
        print("🔍 Đọc Scene PLY...")
        scene = o3d.io.read_point_cloud(scene_ply_path)

        print(f"📌 Scene Points: {len(scene.points)}")
        print(f"📌 ROI Points (trước align): {len(roi_pcd.points)}")

        # ===== Chuyển ROI sang hệ tọa độ Scene =====
        roi_pcd = self.align_roi_to_scene(roi_pcd)
        print(f"✅ ROI Points (sau align): {len(roi_pcd.points)}")

        # KDTree cho Scene (tăng tốc độ tìm kiếm)
        kdtree = o3d.geometry.KDTreeFlann(scene)

        scene_points = np.asarray(scene.points)
        keep_mask = np.zeros(len(scene_points), dtype=bool)
        roi_points = np.asarray(roi_pcd.points)

        print("⏳ Đang lọc Scene theo ROI...")

        for roi_pt in roi_points:
            _, idx, dist = kdtree.search_knn_vector_3d(roi_pt, 1)
            if len(idx) > 0 and dist[0] < self.distance_threshold ** 2:
                keep_mask[idx[0]] = True

        filtered_scene = scene.select_by_index(np.where(keep_mask)[0])
        print(f"🎯 Filtered Points: {len(filtered_scene.points)}")

        if len(filtered_scene.points) == 0:
            print("⚠️ Không tìm được điểm nào thuộc ROI trong Scene!")

        # Lưu kết quả
        o3d.io.write_point_cloud("filtered_roi.ply", filtered_scene)
        print("💾 Đã lưu: filtered_roi.ply")

        return filtered_scene

Test thử với 1 ảnh

In [5]:
# scene_path = "Public data/Public data train/ply/0002.ply"
# roi_path = "output/filtered_roi_0002.ply"

# roi = o3d.io.read_point_cloud(roi_path)
# filterer = PointCloudFilter(distance_threshold=0.01)

# filtered = filterer.filter_scene_with_roi(scene_path, roi)
# o3d.io.write_point_cloud("output/filtered_scene_0002.ply", filtered)

# print("✅ Filtered PLY saved: output/filtered_scene_0002.ply")

In [16]:
scene_path = "./Public data/Public data train/ply/0018.ply"
roi_path = "./data/roi_pointclouds/0002.ply"
roi_pcd = o3d.io.read_point_cloud(roi_path)  # ROI từ depth

pcf = PointCloudFilter(distance_threshold=0.01)
filtered = pcf.filter_scene_with_roi(scene_path, roi_pcd)

o3d.visualization.draw_geometries([filtered])


🔍 Đọc Scene PLY...
📌 Scene Points: 675992
📌 ROI Points (trước align): 95860
✅ ROI Points (sau align): 95860
⏳ Đang lọc Scene theo ROI...
🎯 Filtered Points: 24791
💾 Đã lưu: filtered_roi.ply


Xử lý trên cả tập data

In [17]:
import os

In [18]:
# 1. Cấu hình
scene_folder = "D:/AI_project/VAR/Public data/Public data train/ply"       # thư mục chứa các file scene PLY
roi_folder = "./data/roi_pointclouds"          # file ROI PCD
output_folder = "./data/filtered_ply"    # thư mục lưu kết quả
distance_threshold = 0.01

os.makedirs(output_folder, exist_ok=True)


# 3. Tạo object filter
pc_filter = PointCloudFilter(distance_threshold)

# 4. Lặp qua tất cả file PLY trong thư mục
for filename in os.listdir(scene_folder):
    if not filename.endswith(".ply"):
        continue

    scene_path = os.path.join(scene_folder, filename)
    roi_path = os.path.join(roi_folder, filename)

    print(f"\nĐang xử lý: {filename}")
    print(f"Scene path: {scene_path}")
    print(f"ROI path:   {roi_path}")

    if not os.path.exists(scene_path):
        print("Không tìm thấy scene file, bỏ qua.")
        continue
    if not os.path.exists(roi_path):
        print("Không tìm thấy ROI file, bỏ qua.")
        continue

    roi_pcd = o3d.io.read_point_cloud(roi_path)
    if roi_pcd is None or len(roi_pcd.points) == 0:
        print("Không đọc được hoặc ROI rỗng.")
        continue

    filtered_scene = pc_filter.filter_scene_with_roi(scene_path, roi_pcd)

    if filtered_scene is None or len(filtered_scene.points) == 0:
        print("Không có điểm hợp lệ, bỏ qua.")

    output_path = os.path.join(output_folder, filename)
    success = o3d.io.write_point_cloud(output_path, filtered_scene)
    print("Kết quả lưu:", "Thành công" if success else "Thất bại")


Đang xử lý: 0000.ply
Scene path: D:/AI_project/VAR/Public data/Public data train/ply\0000.ply
ROI path:   ./data/roi_pointclouds\0000.ply
🔍 Đọc Scene PLY...
📌 Scene Points: 681218
📌 ROI Points (trước align): 96159
✅ ROI Points (sau align): 96159
⏳ Đang lọc Scene theo ROI...
🎯 Filtered Points: 25534
💾 Đã lưu: filtered_roi.ply
Kết quả lưu: Thành công

Đang xử lý: 0001.ply
Scene path: D:/AI_project/VAR/Public data/Public data train/ply\0001.ply
ROI path:   ./data/roi_pointclouds\0001.ply
🔍 Đọc Scene PLY...
📌 Scene Points: 679692
📌 ROI Points (trước align): 95676
✅ ROI Points (sau align): 95676
⏳ Đang lọc Scene theo ROI...
🎯 Filtered Points: 23511
💾 Đã lưu: filtered_roi.ply
Kết quả lưu: Thành công

Đang xử lý: 0002.ply
Scene path: D:/AI_project/VAR/Public data/Public data train/ply\0002.ply
ROI path:   ./data/roi_pointclouds\0002.ply
🔍 Đọc Scene PLY...
📌 Scene Points: 684068
📌 ROI Points (trước align): 95860
✅ ROI Points (sau align): 95860
⏳ Đang lọc Scene theo ROI...
🎯 Filtered Points: 2

Xem ảnh

In [23]:
pc = o3d.io.read_point_cloud("D:/AI_project/VAR/data/filtered_ply/0048.ply")
print(pc)
o3d.visualization.draw_geometries([pc])


PointCloud with 31790 points.
